# Portformer API Documentation

This notebook should document all common use cases and workflows.

In [61]:
%load_ext autoreload

In [62]:
%autoreload 2

In [63]:
from portformer.beta import *

## User data

users can upload 
 * **portfolio**:  
   * (portfolio_id, firm_id, user_id, client_id, name, description, tags)
   * (ticker, shares, amount, currency, as_of_date, buckets, tags, note)
 * **trades**:
   * (portfolio_id, firm_id, user_id, client_id, name, description, tags)
   * (ticker, change_shares, amount, currency, price, mid_price?, fees?)
 * **todo** (other tables from the backtesters)

 * **series**: 
   * (series_id, firm_id, user_id, custom_ticker, name, description, tags, source, asset_class)
   * (series_id, as_of_date, high, low, open, close, volume)
 
 * **model**:
   * (model_id, firm_id, user_id, name, description, tags, source)
   * (model_id, as_of_date, series_id, weight)

 * **analysis**:
   * TODO record

## User methods



## Examples - analyze portfolio

  1. User uploads portfolio:  
    * `portfolio = create_portfolio(holdings) or create_portfolio(trades)`
    * If missing data on a ticker, require uploading `custom_series` and repeat step 1

  2. Add Buckets:
    * `portfolio = organize_portfolio(portfolio, window=252, as_of_date=None)`
    
  3. Create Model:
     * `model = create_model_from_portfolio(portfolio, min_distance=1)`
     * Creates a model from the buckets provided a min_distance between nodes
 
  4. Analyze Portfoio:
    * `stats = analyze_portfolio(portfolio, window=252*5, rolling=22, as_of_date=None)`
    * returns stats for each bucket and holding (over total window and on a rolling basis)
   
  5. Replicate: for each node in tree, score and replicate
    * `passive_alternatives = portformer_passive_replacements(holdings, os_date, universe_filters=None)`
    * `active_alternatives = portformer_active_replacements(holdings, os_date, universe_filters=None)`
    * `portfolio_alternatives = replicate_portfolio(portfolio, stats.navs, os_date, universe='etf')`

  6. Generate Alternatives: 
    * `new_portfolios, proposed_changes = propose_optimized(portfolio, candidates=[passive_alternatives, active_alternatives, portfolio_alternatives], batch=10, priorities=[('fees','desc'), ('drawdown', 1)], feedback=None)`
    * Feedback dataframe is a set of relative views between strategies with reasons why (tags, notes)
    * Generate `stats`, `portfolio meta` and `holdings fundamental data`
    
  7. Risk Forecasts:
    * `risk_model = portfolio_risk(portfolio, method='breakpoint')`
    * `risk_model = portfolio_risk(model, method='historical')`

  8. Compare Portfolios:
    * `relative_stats = compare(portfolio, portfolio2, risk_model=None)`
    * `relative_stats = compare(portfolio, model, risk_model=None)`

  9. Align Portfolios
    * `new_portfolio, transactions, costs = align(portfolio, model, risk_model=None, max_trades=None, tolerance=None, distance_metric=None)`
    * The the larger the transactions that bring the portfolios more closely together the greater the costs.

  10. Optimize Portfolio:
    * Run Portfolio optimization up to a certain tree depth.  Size leafs below max_depth to `equal` or `inverse_vol` or `market_cap` or another `field`
    * Given tickers, holdings and bucketing
      * `relative_stats, transactions, costs = compare(portfolio, portfolio2)`

# Robinhood example

```
holdings = get_robin_hood_portfolio()
portfolio = create_portfolio(holdings)

universe = get_universe(asset_class='etf', **filters)  #
universe = filter_universe(universe, **filters)  # filter by price, market cap, volume, etc...

risk_model = create_risk_model(universe, method='breakpoint')
model = optimize(universe, risk_model=risk_model, method='hrp')

relative_stats = compare(portfolio, model, risk_model=risk_model)

new_portfolio, txns, costs = align(
    portfolio, model, risk_model=risk_model, max_trades=None, tolerance=None, distance_metric='vol_contrib'
)

# Execute txns
execute_robin_hood_trades(txns)
```

# Friend's Neural Net Strategy Example
```
custom_indexes = {}
for name in total_returns.columns:
    custom_indexes[name] = upload_index(
        closes=total_returns[name], 
        names=name, 
        descriptions=None, 
        tags=None, 
        source='Neural Network',
        asset_class=None
    )
total_returns = total_returns.rename(columns=custom_indexes)
universe = total_returns.columns.tolist()

risk_model = create_risk_model(universe, method='historical')
model = optimize(universe, risk_model=risk_model, method='equal')

# Bucket
portfolio = initialize_portfolio(model, capital=1_000_000)
portfolio = organize_portfolio(portfolio, window=252, as_of_date=None)
holdings = portfolio.holdings

portfolio_alternatives = replicate_portfolio(portfolio, stats.navs, os_date, universe='etf')

# view tree colored by metric
# color based on improvement in portfolio stat if its substitutute

# Greedy portfolio search
portfolio_alternatives_ranked = rank_alternatives_by_tree(
    portfolio,
    candidates=portfolio_alternatives, 
    algo='gready', 
    metric='sharpe'
)

new_portfolios, proposed_changes = propose_optimized(
    portfolio, 
    candidates=portfolio_alternatives, 
    batch=10, 
    priorities=[('fees','desc'), ('drawdown', 1)],
    feedback=None
)
```

# Rose.AI / Quiver Example
```
signal = get_quiver_sentiment_metric_by_ticker()
universe = signal.columns.tolist()

base_risk_model = create_risk_model(universe, method='historical')
risk_model = base_risk_model.copy()

# Create trading factor from sentiment
factor = (signal.rolling(window=2).diff() > 0) * 1

# Update forecasts
risk_model.mu *= factor

# Create weights
base_model = optimize(universe, risk_model=base_risk_model, method='iv')
model = optimize(universe, risk_model=risk_model, method='iv')

# View Stats
base_stats = analyze_portfolio(base_model, window=252*5, rolling=22, as_of_date=None)
stats = analyze_portfolio(model, window=252*5, rolling=22, as_of_date=None)
relative_stats = compare(base_model, model, risk_model=base_risk_model)

```


# Volos Integration

The ability to generate a sector rotation + `SPY/AGG` strategies

```
universe = ['SPY', 'AGG']
# or
universe = ['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']

risk_model = create_risk_model(universe, method='breakpoint')
model = optimize(universe, risk_model=risk_model, method='hrp')

model_id = save_model(model)

model = get_model(model_id)
portfolio = initialize_portfolio(model, capital=1_000_000)
stats = analyze_portfolio(portfolio, window=252*5, rolling=22, as_of_date=None)

weights = get_model_weights(model_id, ref_period)

# Execute txns
```

# Backtesting Example

Given Tradable Universe and Collection of trading signals, optimize portfolio construction and trading rules to best separate signal from noise

```
tradable_univese = Universe(**)

technical_signals = Technical('moving_average', **)
custom_data = pd.DataFrame(**)

signals = Factors(exodg=[custom_data], endog=[technical_signals])
transforms = FactorTransform(signals, **)

fcasts = Forecasts(y=universe, x=transforms, **)

risk_model = RiskModels(universe=tradable_universe, forecasts=fcasts, **)

model_portfolios = optimize(risk_model, initial_portfolio)

backtest = Backtest(
    model_portolio.weights,
    initial_portfolio,
    align_portfolio_function
)

stats = backtest

# Determine Statistical robustness by repeating with a permuted
# tradeable universe

# Determine alternatives and max fees can charge
passive_alternatives = get_alternatives(backtest)

# 

```


# Implementation


In [16]:
import pandas as pd
tickers =['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU']



,as_of_date,XLC,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLRE,XLK,XLU
as_of_date,ticker,,,,,,,,,,,


In [60]:
from portformer.beta.risk_model import create_risk_model

universe = [
    "XLC",
    "XLY",
    "XLP",
    "XLE",
    "XLF",
    "XLV",
    "XLI",
    "XLB",
    "XLRE",
    "XLK",
    "XLU",
]

risk_model = create_risk_model(
    tickers=universe,
    method="breakpoint",
    date_range_start="2020-01-01",
    date_range_end="2020-10-10",
)
risk_model


RiskModel(breakpoint) - 11 tickers <BusinessDay> (2020-01-01 00:00:00, 2020-10-09 00:00:00)

In [17]:
model = optimize(universe, risk_model=risk_model, method='hrp')

model_id = save_model(model)

model = get_model(model_id)
portfolio = initialize_portfolio(model, capital=1_000_000)
stats = analyze_portfolio(portfolio, window=252*5, rolling=22, as_of_date=None)

weights = get_model_weights(model_id, ref_period)

NameError: name 'optimize' is not defined